# Lets build out first Deep Neural Network

All dependencies for this notebook is listed in the requirements.txt file. One parent above the nbs directory. This list will keep changing as we add to it so be sure to rerun this line after every git pull

In [ ]:
!pip install -r ../requirements.txt

Lets declare our imports

In [115]:
import numpy as np
import torch
from torch import nn
from tqdm import tqdm
import math

In [ ]:
! pip install -q kaggle

In [ ]:
from google.colab import files

In [ ]:
files.upload()

In [ ]:
 ! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets list

In [ ]:
! kaggle competitions download -c 'heart-disease-uci'

In [127]:
class MyFirstNeuralNetwork(torch.nn.Module):
    def __init__(self, in_size=2, out_size=1, hidden_size=3):

        super(MyFirstNeuralNetwork, self).__init__()

        # Set the dimensionality of the network
        self.input_size = in_size
        self.output_size = out_size
        self.hidden_size = hidden_size

        # Initialize our weights
        self._init_weights()

    '''
    Initialize the weights
    '''
    def _init_weights(self):
        # Create an input tensor of shape (2,3)
        self.W_Input = torch.randn(self.input_size, 
                                   self.hidden_size)

        # Create an output tensor of shape (3, 1)
        self.W_Output = torch.randn(self.hidden_size, self.output_size)
        
        print('Output shape is: {}'.format(self.W_Output.shape))

    '''
    Create the forward pass
    '''
    def forward(self, inputs):
        # Lets get the element wise dot product
        self.z = torch.matmul(inputs, self.W_Input)
        # We call the activation
        self.state = self._activation(self.z)
        # Pass it through the hidden layer
        self.z_hidden = torch.matmul(self.state, self.W_Output)
        # Finally activate the output
        output = self._activation(self.z_hidden)

        # Return the output
        return output

    '''
    Backpropagation algorithm implemented
    '''
    def backward(self, inputs, labels, output):
        # What is the error in output
        self.loss = labels - output

        # What is the delta loss based on the derivative
        self.loss_delta = self.loss * self._derivative(output)
        
        # Get the loss for the existing output weight
        self.z_loss = torch.matmul(self.loss_delta, torch.t(self.W_Output))

        # Compute the delta like before
        self.z_loss_delta = self.z_loss * self._derivative(self.state)

        # Finally propogate this to our existing weight tensors to update
        # the gradient loss
        self.W_Input += torch.matmul(torch.t(inputs), self.z_loss_delta)
        self.W_Output += torch.matmul(torch.t(self.state), self.loss_delta)

    '''
    Here we train the network
    '''
    def train(self, inputs, labels):
        # First we do the foward pass
        outputs = self.forward(inputs)
        # Then we do the backwards pass
        self.backward(inputs, labels, outputs)

    '''
    Here we perform inference
    '''
    def predict(self, inputs):
        pass

    '''
    Here we save the model
    '''
    def save(self):
        pass
    
    '''
    Our non-linear activation function
    '''
    def _activation(self, s):
        # Lets use sigmoid
        return 1 / (1 * torch.exp(-s))

    '''
    Our derivative function used for backpropagation
    Usually the sigmoid prime
    '''
    def _derivative(self, s):
        # derivative of sigmoid
        return s * (1 - s)

In [128]:
nn = MyFirstNeuralNetwork()

Output shape is: torch.Size([3, 1])


In [129]:
labels= torch.tensor(([1],[0],[0],[1]), dtype=torch.float)
inputs = torch.tensor(([20, 90],[10, 20],[30, 40],[20, 50]), dtype=torch.float)

In [84]:
predicted = nn.forward(torch.tensor([20,90], dtype=torch.float))